# Plotting HURDAT2 Storms Alongside Drifter Tracks

In this notebook we will walk through how we can use the [`clouddrift`](https://github.com/Cloud-Drift/clouddrift) library to plot NOAA GDP drifter tracks colored by sea surface temperature, alongside storm tracks from the HURDAT2 dataset, with both datasets directly available from the library. HURDAT2 is a dataset that contains storm track data (including pressure, wind speed, etc...) for the time period 1852 to 2022, both from the North Pacific and North Atlantic ocean basins. The HURDAT2 dataset is originally available from [NOAA AOML](https://www.aoml.noaa.gov/hrd/hurdat/Data_Storm.html), and such is the [NOAA GDP dataset](https://www.aoml.noaa.gov/phod/gdp/index.php).

This notebook proceeds in steps by:
 1. Access and open the necessary datasets,
 2. Subset these datasets by selecting their data for a given year and region,
 3. Generate an animation showing as a function of time the selected data.

### 0. Library import

In [ ]:
# useful library imports
import xarray as xr
import numpy as np
import pandas as pd
import clouddrift as cd
from clouddrift.ragged import unpack

from datetime import datetime

# plotting and mapping imports
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.cm as cm
import matplotlib.colors as colors
import cartopy.crs as ccrs
from mpl_toolkits.axes_grid1 import make_axes_locatable


### 1. Dataset imports

We access and open as an xarray dataset a ragged array version of the [six-hourly NOAA Global Drifter Program dataset](https://www.aoml.noaa.gov/phod/gdp/interpolated/data/all.php) which is hosted as an experiment in an AWS S3 bucket. Please consult the webpage of the dataset to investigate alternative ways of accessing these data. Below we also provide a couple of alternative ways of accessing these data. 

In [ ]:
# dataset URL on the AWS S3 bucket:
url = "https://noaa-oar-hourly-gdp-pds.s3.amazonaws.com/experimental/gdp6h_ragged_sep23.zarr"
drifter_ds = xr.open_dataset(url, decode_times=False, engine="zarr")
# re-assign and rename the ID variable to follow clouddrift conventions for ragged array datasets:
drifter_ds = drifter_ds.rename_vars({"ID": "id"}).assign_coords({"id": drifter_ds.ID}).drop_vars(["ids"])

# alternatively, access the ragged array 6-hourly dataset from the AOML website via the clouddrift API with the following command:
# drifter_ds = cd.datasets.gdp6h()

# alternatively, access the individual netcdf files from the AOML FTP server and aggegrate locally these data into a ragged array dataset via the cloudrift API with the following command:
# drifter_ds = cd.adapters.gdp6h.to_raggedarray().to_xarray()

drifter_ds

Next we access the HURDAT2 dataset directly using the clouddrift API. The command below looks for the dataset locally and if it is not found the original data are downloaded and aggregated into a ragged array dataset which is then opened as an xarray dataset.

In [ ]:
storm_ds = cd.datasets.hurdat2(decode_times=True)
storm_ds

### 2. Subsample the datasets

Next we select specific subsets of the opened datasets. For this, we use the clouddrift `subset` function provided through the `ragged` module which contains helpful utility functions for working with ragged array data structures. Here we choose to subset storms whose track lied within the Atlantic Ocean near the east coast of North America and was observed between August and October of 2020 (feel free to change any of this selection when running this notebook).  The `subset` function requires to define a criteria as a dictionary as follows:

In [ ]:
# define a time range
year = 2020
start_dt, end_dt = datetime(year, 8, 1), datetime(year, 10, 1)
# define the criteria for the drifter and storm datasets; please note that the times are encoding differently in the two datasets and as such we need to specify the time range differently for each criteria.

drifter_criteria = dict(
    lat=(10, 50),
    lon=(-80, -20), 
    time=(
        start_dt.timestamp(),
        end_dt.timestamp()
    )
)

storm_criteria = criteria = dict(
    lat=(10, 50),
    lon=(-80, -20), 
    time=(
        np.datetime64(int(start_dt.timestamp()), "s"),
        np.datetime64(int(end_dt.timestamp()), "s")
    )
)

We can now use the `subset` function and apply the criteria to both datasets. The `subset` function requires to provide the name of the *row* dimension for the ragged array datasets (see the [clouddrift documentation](https://clouddrift.org)), which is `traj` in both cases.

In [ ]:
matching_storms = cd.ragged.subset(storm_ds, storm_criteria, row_dim_name="traj")
matching_storms

In [ ]:
matching_drifters = cd.ragged.subset(drifter_ds, drifter_criteria, row_dim_name="traj")
matching_drifters

### 3. Plotting and Animation.

Subsetting the datasets above, you should have found 14 storms and 248 drifters. We will now proceed to create a map and some animation to visualize the selected data.

In [ ]:
# create a blank figure with the desired size and resolution
DPI = 384
fig = plt.figure(figsize=(7.75, 4.75), dpi=DPI)
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())
ax.set_extent([-100, 0, 0, 80], crs=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
# why can't we plot let's say all the drifter trajectories?
# line1 = cd.plotting.plot_ragged(ax,
#                                 matching_drifters["lon"], 
#                                 matching_drifters["lat"], 
#                                 rowsize=matching_drifters["rowsize"],
#                                 colors=np.arange(len(matching_drifters["rowsize"])),
#                                 transform=ccrs.PlateCarree(),
#                                 cmap="Blues",)
datetime_label = ax.text(-115, 40, start_dt.strftime('%Y-%m-%d %H:%M:%S'), 
    fontsize=10, 
    color="red", 
    transform=ccrs.PlateCarree(), 
    bbox=dict(facecolor="white", alpha=0.5, edgecolor="none")
)
ax.set_title("Hurricane Season"+str(year))

Were going to iterate over both, the selected storms and the selected drifters. For each of the trajectories, we plot their starting point and store some data to be utilized for generating the animation.

Lets unpack the data variables from a ragged array (1-d array composed of each rows data variable segment) into a list of row data variable segments.

In [ ]:
# create indices for the number of storms and the number of drifters selected
storm_indices = np.arange(0,len(matching_storms["id"]))
drifter_indices = np.arange(0,len(matching_drifters["id"]))

To generate the animation we need to work on the data of each individual storm and each individual drifter. Because these datasets are organized as ragged arrays, we use the clouddrift `unpack` function to generate lists of data. The `unpack` function takes as arguments a ragged array input and a *rowsize* variable that indicates the size of each row of the ragged array.

In [ ]:
# unpack the lat and lon data for the storms
storm_lons = unpack(matching_storms.lon, matching_storms.rowsize)
storm_lats = unpack(matching_storms.lat, matching_storms.rowsize)

# unpack the lat, lon, and sst data for the drifters
drifter_lons = unpack(matching_drifters.lon, matching_drifters["rowsize"])
drifter_lats = unpack(matching_drifters.lat, matching_drifters["rowsize"])
drifter_temps = unpack(matching_drifters.temp, matching_drifters["rowsize"])

We now use the unpacked rows and plot the initial starting point of the storm and drifter trajectories (we also do some setup with setting up indices to make searching easier later).

In [ ]:
storm_lines = list()
drifter_lines = list()

for storm_idx in storm_indices:
    selected_lon, selected_lat = storm_lons[storm_idx], storm_lats[storm_idx]
    selected_lon, selected_lat = selected_lon.set_xindex("time"), selected_lat.set_xindex("time")
    line = ax.plot(selected_lon[0], selected_lat[0],
        linestyle="-", linewidth=3,
        transform=ccrs.PlateCarree(),
    )
    storm_lines.append((selected_lon, selected_lat, line[0]))

for drifter_idx in drifter_indices:
    selected_lon, selected_lat, selected_temp = drifter_lons[drifter_idx], drifter_lats[drifter_idx], drifter_temps[drifter_idx]
    selected_lon, selected_lat, selected_temp = (selected_lon.set_xindex("time"), selected_lat.set_xindex("time"), selected_temp.set_xindex("time"))
    line = ax.plot(selected_lon[0], selected_lat[0],
        linestyle="-", linewidth=1,
        transform=ccrs.PlateCarree(),
    )
    drifter_lines.append((selected_lon, selected_lat, selected_temp, line[0]))

Let's now find the upper and lower bounds for the temperature values being plotted. We'll use this to create a colormap to color the drifter trajectories. 

In [ ]:
# find the min and max temperatire for the colorscale/colormap
min_t = np.nanmin([np.nanmin(temp) for (_, _, temp, _) in drifter_lines])
max_t = np.nanmax([np.nanmax(temp) for (_, _, temp, _) in drifter_lines])

cmap = plt.get_cmap("inferno")
norm = colors.Normalize(vmin=min_t, vmax=max_t)

print((min_t, max_t, cmap(norm(80))))

We can now add the legend for the colorbar:

In [ ]:
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size="3%", pad=0.75, axes_class=plt.Axes)
fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax, label="temperature (K)")
fig

Let's take the start and end date we've used for the data criteria and generate an range of values that each map uniquely to a frame.

In [ ]:
frame_count = 500
daterange = pd.date_range(start_dt, end_dt, frame_count)

Let's now generate each frame by selecting the date associated to it which we utilize to update each trajectory with new observations

In [ ]:
storm_frames = list()
drifter_frames = list()
frames = dict()
tail_len = 50

for idx, dt in enumerate(daterange):
    if ((idx+1) % 100 == 0):
        print(f"generating index: {idx}")

    storm_updates = list()
    for s_lon, s_lat, s_line in storm_lines:
        sel_d_lon = s_lon.sel(time=slice(start_dt, dt))
        sel_d_lat = s_lat.sel(time=slice(start_dt, dt))
        storm_updates.append((sel_d_lon, sel_d_lat, s_line))

    drifter_updates = list()
    for d_lon, d_lat, d_temp, d_line, in drifter_lines:
        sel_d_lon = d_lon.sel(time=slice(start_dt.timestamp(), dt.timestamp()))
        sel_d_lat = d_lat.sel(time=slice(start_dt.timestamp(), dt.timestamp()))
        sel_d_temp = d_temp.sel(time=slice(start_dt.timestamp(), dt.timestamp()))
        sel_d_lon = sel_d_lon.tail(obs=tail_len)
        sel_d_lat = sel_d_lat.tail(obs=tail_len)
        sel_d_temp = sel_d_temp.tail(obs=tail_len)
        drifter_updates.append((sel_d_lon, sel_d_lat, sel_d_temp, d_line))

    frames[dt] = dict(drifter_updates=drifter_updates, storm_updates=storm_updates)

Next, we define an update function that, using the frame index, selects the frame and updates each trajectories longitude and latitude (and also temperature for the case of the drifters).

In [ ]:

sorted_dates = sorted(frames.keys())

def update(frame_idx):
    frame_dt = sorted_dates[frame_idx]
    frame = frames[frame_dt]
    drifter_updates = frame["drifter_updates"]
    storm_updates = frame["storm_updates"]

    datetime_label.set_text(frame_dt.strftime('%Y-%m-%d %H:%M:%S'))

    updated_lines = list()
    for x_update, y_update, line in storm_updates:
        line.set_xdata(x_update)
        line.set_ydata(y_update)
        updated_lines.append(line)

    for x_update, y_update, temps, line in drifter_updates:
        line.set_xdata(x_update)
        line.set_ydata(y_update)
        if len(temps.data) > 0:
            line.set_color(cmap(norm(np.nanmean(temps))))
        updated_lines.append(line)
    return updated_lines

ani = animation.FuncAnimation(fig=fig, func=update, frames=frame_count, interval=50)

Finally, let's generate the animation!

In [ ]:
import warnings
warnings.filterwarnings("ignore") # bad practice and should never be used but helps us keep the output clean and should only ever be used in experimental/sample code.

ani.save("storm_drifters.gif", dpi=DPI, progress_callback=lambda i, n: print(f'Saving frame {i}/{n}'))

Your working directory should now contains the `storm_drifters.gif` file. Enjoy!